# Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
import pickle
import numpy as np
import pandas as pd
import os

# Data

In [30]:
data = [["layer1", "layer2", "layer3", "target"]]

In [29]:
def getDividedArray(array):
  '''
  Breaks longer arrays into chunks of length four
  :param array: array of layer sequence
  :return array containing chunks of length four
  :rtype: array
  '''
  temp = []
  if len(array) > 4:
    for i in range(len(array)-3):
      temp.append(array[i:i+4])
  return temp


def unpickleData(filename):
  '''
  Get data from pickled files
  :param filename: pickle file with model layer sequence
  :param data: the array containing the model array
  '''
  fileObj = open(filename,'rb')
  array = pickle.load(fileObj)
  if len(array) > 4:
    array = getDividedArray(array)
    for i in array:
      data.append(i)
  else:
    data.append(array)
  fileObj.close()
  return data

In [32]:
for file in os.listdir('../data/models'):
    filename=os.path.join('../data/models',file)
    unpickleData(filename)
    break

In [33]:
print(data)

[['layer1', 'layer2', 'layer3', 'target'], ['Conv2D', 'Conv2D', 'Conv2D', 'MaxPooling2D'], ['Conv2D', 'Conv2D', 'MaxPooling2D', 'Flatten'], ['Conv2D', 'MaxPooling2D', 'Flatten', 'Dense'], ['MaxPooling2D', 'Flatten', 'Dense', 'Dense'], ['Flatten', 'Dense', 'Dense', 'Dense'], ['Dense', 'Dense', 'Dense', 'Dense']]


In [34]:
# Column Names
# data = [["layer1", "layer2", "layer3", "target"]]

# get all data in data array
# connect to ec2 and call unpickleData


# convert it to csv
np.savetxt("../data/csv/data.csv", data, delimiter=",", fmt='%s')

In [2]:
df=pd.read_csv("../data/csv/data.csv")
df.head()

,layer1,layer2,layer3,target
0,Conv2D,Conv2D,Conv2D,MaxPooling2D
1,Conv2D,Conv2D,MaxPooling2D,Flatten
2,Conv2D,MaxPooling2D,Flatten,Dense
3,MaxPooling2D,Flatten,Dense,Dense
4,Flatten,Dense,Dense,Dense


In [6]:
layer_map={
    'Conv2D':0,
    'Dense':1,
    'LSTM':2,
    'SimpleRNN':3,
    'Dropout':4,
    'Flatten':5,
    "ZeroPadding2D":6,
    "AveragePooling2D":7,
    "MaxPooling2D":8   
}

In [9]:
def one_hot_encode(data,label_map):
    '''
    One hot encode the data according to the label map
    :param data: the data to be encoded
    :return: the encoded data
    '''
    one_hot_encoded=list()
    for i in range(len(data)):
        temp=[0]*len(label_map)
        label=df.iloc[i].target
        temp[label_map[label]]=1
        one_hot_encoded.append(temp)
    return pd.DataFrame(one_hot_encoded,columns=label_map.keys())

In [10]:
df2=one_hot_encode(df,layer_map)
df2.head()

,Conv2D,Dense,LSTM,SimpleRNN,Dropout,Flatten,ZeroPadding2D,AveragePooling2D,MaxPooling2D
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0


# Model

Refrenece for the model
[link](https://stackoverflow.com/questions/64209325/if-i-want-to-predict-the-next-element-in-a-sequence-of-numbers-what-do-i-need-t)

For survey:
- [Making Predictions with Sequences](https://machinelearningmastery.com/sequence-prediction/)
- 

In [ ]:
model = Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(9, activation='softmax'),
])

# Prediction and analysis